In [3]:
import torch
torch.__version__

'2.3.1+cu118'

In [4]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

## Downloading The Data

In [5]:
traing_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|█████████████████████████████████████████████████████████████████| 26421880/26421880 [00:07<00:00, 3628681.67it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|█████████████████████████████████████████████████████████████████████████| 29515/29515 [00:00<00:00, 82945.87it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|███████████████████████████████████████████████████████████████████| 4422102/4422102 [00:02<00:00, 1543425.98it/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|███████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<00:00, 28271.25it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [6]:
batch_size = 64
train_dataloader = DataLoader(traing_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [7]:
train_dataloader

## To check image pixels

In [9]:
# for x, y in test_dataloader:
#     print(x, y)

In [11]:
device = "cuda" if torch.cuda.is_available() else 'cpu'   # Autometicly set gpu or cpu
device

'cuda'

## Define Our Custome Model

In [15]:
class NeuralNetworks(nn.Module):
    def __init__(self):
        super(NeuralNetworks, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [16]:
model = NeuralNetworks().to(device)
print(model)

NeuralNetworks(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

## Traing Loop

In [20]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        #compute prediction erro
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagration
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}] ")
        

## test loop

In [24]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            
            X, y = X.to(device), y.to(device)

            #compute prediction erro
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, avg loss: {test_loss:8f} ")
    
        
        

In [25]:
## Start The tring

epochs = 10
for t in range(epochs):
    print(f"Epoch {t + 1}\n------------------------------- ")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Traing Done")

Epoch 1
------------------------------- 
loss: 1.922453 [    0/60000] 
loss: 1.904216 [ 1600/60000] 
loss: 1.785156 [ 3200/60000] 
loss: 1.834580 [ 4800/60000] 
loss: 1.739208 [ 6400/60000] 
loss: 1.666416 [ 8000/60000] 
loss: 1.696552 [ 9600/60000] 
loss: 1.588772 [11200/60000] 
loss: 1.607755 [12800/60000] 
loss: 1.506827 [14400/60000] 
Test Error: 
 Accuracy: 61.4%, avg loss: 1.534271 
Epoch 2
------------------------------- 
loss: 1.595600 [    0/60000] 
loss: 1.569122 [ 1600/60000] 
loss: 1.412953 [ 3200/60000] 
loss: 1.495320 [ 4800/60000] 
loss: 1.390973 [ 6400/60000] 
loss: 1.361906 [ 8000/60000] 
loss: 1.385978 [ 9600/60000] 
loss: 1.297416 [11200/60000] 
loss: 1.331980 [12800/60000] 
loss: 1.235545 [14400/60000] 
Test Error: 
 Accuracy: 62.9%, avg loss: 1.268277 
Epoch 3
------------------------------- 
loss: 1.343830 [    0/60000] 
loss: 1.330438 [ 1600/60000] 
loss: 1.157829 [ 3200/60000] 
loss: 1.274546 [ 4800/60000] 
loss: 1.164225 [ 6400/60000] 
loss: 1.165047 [ 8000/600